In [10]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average

In [11]:
from tensorflow.keras.applications import MobileNetV2, Xception,EfficientNetB0
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.efficientnet import preprocess_input
import tensorflow as tf

In [12]:
train_dir = r'D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset\train'
val_dir =r'D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset\val'
test_dir=r'D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset\test'

In [13]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [14]:
train_images = train_generator.flow_from_directory(
    directory=train_dir,
    #x_col='Filepath',
    #y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    seed=42
    #subset='training'
)

val_images = train_generator.flow_from_directory(
    directory=val_dir,
    #x_col='Filepath',
    #y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    seed=42
    #subset='validation'
)

test_images = test_generator.flow_from_directory(
    directory=test_dir,
    #x_col='Filepath',
    #y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False
)

Found 4758 images belonging to 6 classes.
Found 1020 images belonging to 6 classes.
Found 1020 images belonging to 6 classes.


In [16]:
model_1 = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Model_Skripsi_NEWW (Layer)\mobilenetv2_Percobaan 4(3).h5")
model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='mobilenetv2')
model_2 = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Model_Skripsi_NEWW (Layer)\Percobaan2(1)_Xception.h5")
model_2 = Model(inputs=model_2.inputs,
                outputs=model_2.outputs,
                name='xception')
model_3 =  load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Model_Skripsi\Percobaan3(2)_EfficientNetB0.h5")
model_3 = Model(inputs=model_3.inputs,
                outputs=model_3.outputs,
                name='efficientnetb0')
models = [model_1, model_2,model_3]
model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input)[0] for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

In [17]:
import tensorflow as tf
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [18]:
ensemble_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        f1_score,
    ]
)

In [19]:
history = ensemble_model.fit(
    train_images,
    validation_data=val_images,
    epochs=300,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
             restore_best_weights=True
        )
    ]
)

Epoch 1/300


d:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/75 ━━━━━━━━━━━━━━━━━━━━ 21:12 20s/step - accuracy: 0.8836 - auc: 0.9304 - f1_score: 0.4726 - loss: 0.8839 - precision: 0.9591 - recall: 0.3149

In [ ]:
ensemble_model.evaluate(test_images)

16/16 ━━━━━━━━━━━━━━━━━━━━ 134s 8s/step - accuracy: 0.9208 - auc: 0.9486 - f1_score: 0.7051 - loss: 0.7647 - precision: 0.9085 - recall: 0.5834


[0.6812148094177246,
 0.9295752048492432,
 0.9213161468505859,
 0.6313725709915161,
 0.9606459140777588,
 0.741193950176239]

In [ ]:
ensemble_model.save_weights('enmodel.h5')


In [ ]:
# Recreate the model structure and load the weights
ensemble_model.load_weights('enmodel_1.h5')

In [ ]:
print(ensemble_model.summary())
print(type(ensemble_model))


Model: "ensemble"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mobilenetv2         │ (None, 6)         │  2,439,238 │ input_layer_1[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ xception            │ (None, 6)         │ 21,141,038 │ input_layer_1[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb0      │ (None, 6)         │  4,230,825 │ input_layer_1[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_1 (Average) │ (None, 6)         │          0 │ mobilenetv2[0][0… │
│                     │                   │            │ xception[0][0],   │
│                     │                   │            │ efficientnetb0[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 29,095,235 (110.99 MB)

 Trainable params: 642,066 (2.45 MB)

 Non-trainable params: 27,169,035 (103.64 MB)

 Optimizer params: 1,284,134 (4.90 MB)

None
<class 'keras.src.models.functional.Functional'>


In [ ]:
ensemble_model.save('saved_model_path', save_format='tf')


ValueError: The `save_format` argument is deprecated in Keras 3. Please remove this argument and pass a file path with either `.keras` or `.h5` extension.Received: save_format=tf

In [ ]:
import pickle
#Model Save
#ensemble_model.save_weights('enmodel_weights.h5')
ensemble_model.save('enmodel_keras2.h5')

TypeError: cannot pickle 'module' object